In [241]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import csv
import os
import re

measurements = ["oz", "ml", "lb", "l", "ounces", "g"]
numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9","0", "."]
IN_FILE_PATH = os.path.join("..//data", "items.txt")
OUTPUT_DIR = "data"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, 'ingredient_prices.csv')

In [275]:
def load_ingredients():
    """Loads the list of ingredients to a list"""

    df = query_and_reshape_long()
    recoded = recode_long_data(df)
    summary = recoded[["ingredient", "amount"]].groupby("ingredient").agg(["mean", "sum"])
    return summary.index.values.tolist()

In [242]:
def read_txt_as_json():
    
    j_list = []
    
    with open(IN_FILE_PATH) as text:
        
        json_list = [line.rstrip("\n") for line in text]
        for j in json_list:
            j_list.append(json.loads(j))
        return j_list
       

In [243]:
def split(word):
    return [char for char in word]

In [268]:
 def get_product_info():
        
        j_list = read_txt_as_json()
        product_dict = {}
        for j in j_list:
            amount_temp = []
            units_temp = []
            price_temp = []
            desc_temp = []
            ingredient = j["request_parameters"]["search_term"]
            key = ingredient
            product_dict.setdefault(key, [])
            a = "search_results"
            if a not in j:
                continue
            else:
                results = j["search_results"]
                i = 0
                for r in results[0:5]:
                    n_temp = []
                    m_temp = []
                    title = r["product"]["title"]
                    chars = split(title)
                    title_list = title.split()
                    price = r["offers"]["primary"]["price"]
                    price_temp.append(price)
                    desc_temp.append(title)
                    for c in reversed(range(len(chars))):
                        if (chars[c] not in (''.join(numbers))) and (len(n_temp) > 0):
                            break
                        if c ==  0:
                            amount_temp.append("NA")
                        if chars[c] not in (''.join(numbers)):
                            continue
                        else:
                            for n in numbers:
                                if n == chars[c]:
                                    n_temp.append(chars[c])
                    amount_temp.append((''.join(n_temp))[::-1])
                    for t in reversed(range(len(title_list))):
                        for m in measurements:
                            if len(m_temp) > 0:
                                break
                            if t == 0:
                                units_temp.append("NA")
                            if m not in (title_list[t].lower()):
                                continue
                            else:
                                m_temp.append(m)
                    units_temp.append(m_temp)
                product_dict[key].append(price_temp)
                product_dict[key].append(amount_temp)
                product_dict[key].append(units_temp)
                product_dict[key].append(desc_temp)
        return product_dict


In [274]:
product_dict = get_product_info()
len(product_dict["ice cream"][0])



5

In [260]:
def drop_bad_products(product_dict):
    for key in product_dict:
        for name in key


In [261]:
drop_bad_products(product_dict)

absinthe
advocaat
aperitif
aquavit
baileys
beer
bitters
bourbon
brandy
butter
cachaca
carbonated water
champagne
cherry heering
club soda
coffee
cognac
cornstarch
drambuie
dry vermouth
dubonnet rouge
egg
erin cream
everclear
flavored rum
flavored vodka
food coloring
frangelico
fruit
fruit juice
galliano
gin
glycerine
grain alcohol
herb
hot sauce
ice
ice cream
jagermeister
liqueur
milk
mix
nut
olive brine
pernod
pisang ambon
pisco
port
prosecco
red wine
ricard
rum
sambuca
sarsaparilla
schnapps
scotch
sherry
soda
soy sauce
spice
sugar
sugard
sweet and sour
tequila
tonic water
vermouth
vodka
water
whiskey
wine
zima


In [114]:
# def json_to_text(jsons, path):
    
    
#     f = open(path, "w+")
#     for i in jsons:
#         f.write(i + "\n")
#     f.close()
# def to_csv(price_dict, path):
# """ Takes the list of (ingredient, price/oz) tuples and writes them to a csv"""

#     with open(OUT_PATH, 'w+') as out_file:
#         csv_writer = csv.writer(out_file)
#         header = ['ingredient', 'price_per_oz']
#         csv_writer.writerow(header)
#         csv_writer.writerows(price_dict)

In [127]:
# if __name__ == '__main__':

#     os.makedirs(OUT_DIR, exist_ok = True)
#     json_to_text(jsons, JSON_PATH)
#     to_csv(item_dict, OUT_PATH)

    